# Recommending Mobile Plans: A Machine Learning Approach for Megaline

by Mikhail Karepov

Megaline, a mobile carrier, wants to help subscribers transition from legacy plans to newer ones: **Smart** or **Ultra**. This project builds a classification model that learns from user behavior — such as number of calls, minutes used, messages sent, and data consumption — to predict which plan is a better fit for each customer.

The goal is to develop the most accurate model (at least **75%** accuracy) to support Megaline’s marketing strategy and improve user experience by offering personalized plan recommendations.

Using Python and scikit-learn, we’ll evaluate several machine learning algorithms and compare their performance.

**Table of contents**<a id='toc0_'></a>    
- 1. [Initialization](#toc1_)    
- 2. [Load Data](#toc2_)    
- 3. [Prepare the Data](#toc3_)    
  - 3.1. [General Info](#toc3_1_)    
    - 3.1.1. [Duplicates](#toc3_1_1_)    
- 4. [Modeling](#toc4_)    
  - 4.1. [Split the Data](#toc4_1_)    
  - 4.2. [Train Models](#toc4_2_)    
  - 4.3. [Evaluate Accuracy](#toc4_3_)    
- 5. [Test the Best Model](#toc5_)    
- 6. [Sanity Check](#toc6_)    
- 7. [Conclusion](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=3
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### 1. <a id='toc1_'></a>[Initialization](#toc0_)

In [46]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### 2. <a id='toc2_'></a>[Load Data](#toc0_)

The dataset contains monthly behavior data for Megaline's subscribers, including:

- Number of calls (`calls`)
- Total call duration in minutes (`minutes`)
- Number of text messages (`messages`)
- Internet traffic used in MB (`mb_used`)
- Current plan (`is_ultra`):  
  - `1` = Ultra  
  - `0` = Smart

In [47]:
df = pd.read_csv('./datasets/users_behavior.csv')
display(df.head())
print()
display(df.info())
print()
display(df.describe())
print()
display(df.sample(10))
print()
df.corr()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


,calls,minutes,messages,mb_used,is_ultra
1974,102.0,747.02,0.0,20964.21,0
2966,128.0,869.07,0.0,21723.95,1
811,36.0,322.57,41.0,12259.44,0
906,49.0,291.00,73.0,15791.45,0
2123,34.0,252.87,26.0,24550.06,1
959,48.0,341.80,53.0,5910.35,1
252,50.0,308.48,51.0,21387.26,0
2157,68.0,399.32,0.0,10204.48,0
1878,156.0,1044.13,149.0,26649.18,1
739,36.0,314.67,16.0,16476.83,0


,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Although the number of calls and call duration is highly correlated (`r ≈ 0.98`), keeping both features resulted in slightly higher model accuracy. 

Random Forests can handle multicollinearity effectively, both were retained to preserve maximum predictive power.

### 3. <a id='toc3_'></a>[Prepare the Data](#toc0_)

#### 3.1. <a id='toc3_1_'></a>[General Info](#toc0_)

- The dataset contains **3,214 entries** and **no missing values**.
- All columns have appropriate data types:
  - `calls`, `minutes`, `messages`, `mb_used`: `float64`
  - `is_ultra`: `int64`
- The target variable `is_ultra` is binary (0 for Smart, 1 for Ultra).

##### 3.1.1. <a id='toc3_1_1_'></a>[Duplicates](#toc0_)

In [48]:
df.duplicated().sum()

0

**No duplicates** were found.

### 4. <a id='toc4_'></a>[Modeling](#toc0_)

#### 4.1. <a id='toc4_1_'></a>[Split the Data](#toc0_)

We split the data into **training**, **validation** and **test** sets:

- 60% of the data is used for training
- 20% is used for validation
- 20% is used for test
- The random seed is fixed to ensure reproducibility

In [49]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

#Create test set (20%)
features_temp, features_test, target_temp, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

#Split remaining 80% into training and validation (60% train, 20% val)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_temp, target_temp, test_size=0.25, random_state=12345)

#### 4.2. <a id='toc4_2_'></a>[Train Models](#toc0_)

We trained and compared the performance of the following classification models:

- **Logistic Regression** – a simple linear model often used as a baseline
- **Decision Tree Classifier** – a model that splits the data based on feature thresholds
- **Random Forest Classifier** – an ensemble of decision trees that improves accuracy and reduces overfitting

Each model was trained on the training set and evaluated on the validation set.

In [50]:
# Logistic Regression
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(features_train, target_train)
lr_preds = lr_model.predict(features_valid)
lr_accuracy = accuracy_score(target_valid, lr_preds)

# Decision Tree
tree_model = DecisionTreeClassifier(random_state=12345)
tree_model.fit(features_train, target_train)
tree_preds = tree_model.predict(features_valid)
tree_accuracy = accuracy_score(target_valid, tree_preds)

# Random Forest
forest_model = RandomForestClassifier(random_state=12345)
forest_model.fit(features_train, target_train)
forest_preds = forest_model.predict(features_valid)
forest_accuracy = accuracy_score(target_valid, forest_preds)

In [51]:
# Display results
print(f"Logistic Regression Accuracy: {lr_accuracy:.3f}")
print(f"Decision Tree Accuracy: {tree_accuracy:.3f}")
print(f"Random Forest Accuracy: {forest_accuracy:.3f}")

Logistic Regression Accuracy: 0.729
Decision Tree Accuracy: 0.712
Random Forest Accuracy: 0.792


**Decision Tree: Hyperparameter Tuning**

To improve the performance of the Decision Tree model, we adjusted its `max_depth` hyperparameter, which controls how deep the tree can grow.

We tested depths from **1 to 10** to find the value that gives the best balance between underfitting and overfitting.

**Random Forest: Hyperparameter Tuning**

Random Forest is an ensemble model that combines multiple decision trees. We tuned the number of trees (`n_estimators`) and tree depth (`max_depth`) to optimize performance.

We ran experiments with:

- `n_estimators`: 10 to 100 (in steps)
- `max_depth`: 5 to 15

By evaluating different combinations, we selected the configuration that gave the best accuracy on the validation set.

In [52]:
# Decision Tree tuning
print("Decision Tree tuning results:")

best_tree_accuracy = 0
best_tree_depth = None

for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print(f"Max depth = {depth}: Accuracy = {accuracy:.3f}")
    
    if accuracy > best_tree_accuracy:
        best_tree_accuracy = accuracy
        best_tree_depth = depth

print(f"Best Decision Tree model: max_depth = {best_tree_depth}, Accuracy = {best_tree_accuracy:.3f}")
print()
# Random Forest tuning
print("Random Forest full tuning results:")
best_forest_accuracy = 0
best_forest_params = ()
for est in range(10, 110, 10):           
    for depth in range(5, 16):           
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        print(f"n_estimators = {est}, max_depth = {depth}: Accuracy = {accuracy:.3f}")
        
        if accuracy > best_forest_accuracy:
            best_forest_accuracy = accuracy
            best_forest_params = (est, depth)

print(f"Best model: n_estimators = {best_forest_params[0]}, max_depth = {best_forest_params[1]}, Accuracy = {best_forest_accuracy:.3f}")

Decision Tree tuning results:
Max depth = 1: Accuracy = 0.739
Max depth = 2: Accuracy = 0.757
Max depth = 3: Accuracy = 0.765
Max depth = 4: Accuracy = 0.764
Max depth = 5: Accuracy = 0.759
Max depth = 6: Accuracy = 0.757
Max depth = 7: Accuracy = 0.774
Max depth = 8: Accuracy = 0.767
Max depth = 9: Accuracy = 0.762
Max depth = 10: Accuracy = 0.771
Best Decision Tree model: max_depth = 7, Accuracy = 0.774

Random Forest full tuning results:
n_estimators = 10, max_depth = 5: Accuracy = 0.778
n_estimators = 10, max_depth = 6: Accuracy = 0.785
n_estimators = 10, max_depth = 7: Accuracy = 0.787
n_estimators = 10, max_depth = 8: Accuracy = 0.785
n_estimators = 10, max_depth = 9: Accuracy = 0.795
n_estimators = 10, max_depth = 10: Accuracy = 0.790
n_estimators = 10, max_depth = 11: Accuracy = 0.788
n_estimators = 10, max_depth = 12: Accuracy = 0.792
n_estimators = 10, max_depth = 13: Accuracy = 0.793
n_estimators = 10, max_depth = 14: Accuracy = 0.788
n_estimators = 10, max_depth = 15: Accur

**Decision Tree: Hyperparameter Tuning**

We tested depths from **1 to 10**. The best result was:

- **Max depth = 7** - **Accuracy = 0.774**

**Random Forest: Hyperparameter Tuning**

We tuned the number of trees (`n_estimators`) and tree depth (`max_depth`) for the Random Forest model.

The best result was:

- **n_estimators = 20**, **max_depth = 15** - **Accuracy = 0.802**

#### 4.3. <a id='toc4_3_'></a>[Evaluate Accuracy](#toc0_)

We used accuracy score as the performance metric. Our goal was at least **75% accuracy** on the validation set.

Below are the validation scores for the **best-tuned versions** of each model:

- **Logistic Regression Accuracy**: `0.729`  
- **Decision Tree Accuracy (max_depth=7)**: `0.774` 
- **Random Forest Accuracy (n_estimators=20, max_depth=15)**: `0.802`

The **Random Forest model** had the highest accuracy and exceeded the target threshold. It was selected for final testing on unseen data.

### 5. <a id='toc5_'></a>[Test the Best Model](#toc0_)

After tuning the models on the validation set, the **Random Forest Classifier** with `n_estimators=20` and `max_depth=15` showed the highest accuracy of **80.2%**. 

Now we’ll test this final model on a **completely untouched test set** (20% of the original dataset). This helps us evaluate how well the model performs on truly unseen data and confirm that it generalizes well.

We proceed by:

- Combining the training and validation sets
- Training the best model on this combined data
- Making predictions on the test set
- Evaluating the test accuracy

Our goal remains: reach at least **75% accuracy** on the test set.

In [53]:
# Combine training and validation data
features_train_val = pd.concat([features_train, features_valid])
target_train_val = pd.concat([target_train, target_valid])

# Retrain the best model on the full training + validation data
final_model = RandomForestClassifier(n_estimators=20, max_depth=15, random_state=12345)
final_model.fit(features_train_val, target_train_val)

# Predict on the untouched test set
test_predictions = final_model.predict(features_test)

# Evaluate performance
test_accuracy = accuracy_score(target_test, test_predictions)
print(f"Final accuracy on test set: {test_accuracy:.3f}")

Final accuracy on test set: 0.784


### 6. <a id='toc6_'></a>[Sanity Check](#toc0_)

We perform a sanity check to make sure our model is working as expected — and not just predicting the most common class or benefiting from random chance.

We’ll compare our final model’s accuracy on the test set against two checks:

- **Random guesser** — simulates predicting randomly between Smart and Ultra
- **Most frequent class** — always predicts the more common plan (e.g., Smart)

If the model outperforms these checks, we can be confident that it actually learned meaningful patterns from the data.

In [54]:
# Check 1: Random guess (0 or 1 with equal probability)
random_preds = np.random.randint(0, 2, size=len(target_test))
random_accuracy = accuracy_score(target_test, random_preds)

# Check 2: Predict the most frequent class in the training+validation set
most_common_class = target_train_val.mode()[0]
constant_preds = np.full_like(target_test, most_common_class)
constant_accuracy = accuracy_score(target_test, constant_preds)

# Show results
print(f"Check (random predictions): {random_accuracy:.3f}")
print(f"Check (most frequent class): {constant_accuracy:.3f}")
print(f"Model (Random Forest) accuracy: {test_accuracy:.3f}")

Check (random predictions): 0.504
Check (most frequent class): 0.695
Model (Random Forest) accuracy: 0.784


### 7. <a id='toc7_'></a>[Conclusion](#toc0_)

In this project, we built a machine learning model to recommend mobile plans (Smart or Ultra) based on user behavior.

After exploring the dataset and training multiple models, we selected a **Random Forest Classifier** with `n_estimators=20` and `max_depth=15` as the final model.

Here’s how it performed:

- **Validation accuracy**: 80.2%
- **Test accuracy**: 78.4%

We also ran a sanity check and compared our model to two baselines:
- Random guessing: ~50.4%
- Most frequent class: 69.5%
- Model: 78.4%

Our model clearly outperformed both, confirming it learned meaningful patterns rather than guessing or relying on class imbalance.

**Final result**: A robust classification model with >75% accuracy, ready to support Megaline in recommending personalized plans to customers.